# Python code for final project PHZ5305: Interaction matrix elements and Hartree-Fock (Nov. 29, 2024)

## Introduction

This code is written for the final proejct of PHZ5305 (Nuclear Physics 1), which will provide the tool for the read the list of single particle states and me2b generated, and solves Hartree-Fock Equation.

Additionally, this code will optimize the me2b values into the value that simultaneously gives binding energies of the $^4 He$, $^{12} C$, $^{16} O$.

## Part 1: Import necessary packages.

`numpy` provides mathematical tools for calculation, such as `power`, `list`, and `array`.

`pandas` provides the reading csv file that contains the experimental informations that will provide the `me2b`.

`scipy` provides the optimization feature of this code.

`random` is imported for giving a random value assigned for the other `me2b` values.

In [1]:
import numpy as np
import pandas as pd
import scipy as scp
import scipy.sparse as sp
from scipy import optimize
from scipy.optimize import minimize
import random
import sys
from IPython.display import clear_output

## Part 2: Class for Generate single-particle states, two-body matrix element calculation, and solve Hartree-Fock.

In this block, the Hartree-Fock equation will be solved via algorithm that intorudced at the class (Oct. 14, 2024) with single-particle states and two-body matrix elements. 

`__init__` gives initial definition for this class

`SP_States` generates `dictionary` of SP states, $\pi 0s1/2$, $\nu 0s1/2$, $\pi 0p3/2$, $\nu 0p3/2$, $\pi 0p1/2$, $\nu 0p 1/2$ in `{index: [n, l, j, mj, tz]}` format.  

`me2b` gets 4 index values for $\alpha$, $\beta$, $\gamma$, and $\delta$.

`Solve_HF` solves Hartree-Fock equation with algorithm

In [2]:
class HF_Machina:
    # Initial definition of states
    def __init__(self):
        self.SP_States()
        self.TB_States()
        
    # Generate Dictionaries of Single-Particle States up to 16O
    def SP_States(self):
        
        # Initialize the important parameters which will be applied to loop #
        list_tz      = [-1/2, 1/2]
        n            = 0
        list_l       = [0, 1]
        dict_list_j  = {
            0: [1/2],
            1: [3/2, 1/2]
        }
        
        # Set the index and empty dictionary
        index        = 0 
        dict_SP      = {}
        
        # Loop for fullfill the dictionary
        for l in list_l:
            for tz in list_tz:
                for j in dict_list_j[l]:
                    mj = -1*j
                    while mj < j+1:
                        dict_SP.update({index: [n,l,j,mj,tz]})
                        index += 1
                        mj += 1
        
        # Output
        self.dict_SP = dict_SP
        return dict_SP

    # Generate List of Two-body states in One-body basis (a,b) for Jpi = 0+
    def TB_States(self):
        
        #Initialize
        NSP = 16
        
        list_TB_ind = []
        
        for a in range(NSP):
            for b in range(NSP):
                # Imply possible conditions to be Jpi = 0+ 
                # ja = jb
                j_cond  = (self.dict_SP[a][2] == self.dict_SP[b][2])
                # mja = -mjb
                mj_cond = (self.dict_SP[a][3] == -1*self.dict_SP[b][3])
                # la = lb (since only possible l = 0 or 1)
                l_cond  = (self.dict_SP[a][1] == self.dict_SP[b][1])
                if j_cond and mj_cond and l_cond: list_TB_ind.append((a,b))
        
        list_TB_ind = list({frozenset(pair) for pair in list_TB_ind})
        list_TB_ind = [tuple(sorted(pair)) for pair in list_TB_ind]
        
        self.list_TB = list_TB_ind
        return list_TB_ind
        
    
    # Code for solving Hartree-Fock equation from core 12C 
    # A is the atomic number of nucleus 
    # Z is the charge of nucleus 
    # input_me2b is a me2b input with form of list [(a,b,c,d,v)]
    def Solve_HF(self, A, Z, input_me2b):
        
        # Prepare Initial Value
        self.A_c      = 12
        self.Z_c      = 6
        self.A        = A
        self.Z        = Z
        self.N        = A-Z
        NSP           = len(self.dict_SP)
        
        # Set list of protons
        list_proton   = []
        list_neutron  = []
        for key, value in self.dict_SP.items():
            if value[4] == -0.5 and len(list_proton) != self.Z: list_proton.append(key)
            elif value[4] == 0.5 and len(list_neutron) != self.N: list_neutron.append(key)
            if len(list_proton) == self.Z and len(list_neutron) == self.N : break
        
        list_A = list_proton+list_neutron
        
        # Prepare me2b
        self.dict_me2b = {(me2b[0], me2b[1], me2b[2], me2b[3]): me2b[4] for me2b in input_me2b}
        
        # Computing single-particle Hamiltonian (SPH)
        SPH = np.array([self.onebody(sp_ind) for sp_ind in range(len(self.dict_SP))])
        
        # Initialize the Coefficient Matrix Coe and Density Matrix Rho 
        Coe = np.eye(NSP)
        Rho = np.zeros((NSP, NSP), dtype=object)

        for i_gam in range(NSP):
            for i_del in range(NSP):
                Rho[i_gam, i_del] = sum(Coe[i_gam,i]*Coe[i_del,i] for i in list_A)
        
        #Rho = np.random.rand(NSP, NSP)
        
        #print(Rho)
        
        # Now, we will do the algorithm that will be iterated until it converges at somewhere or its maximum iteration number
        # Set initial value for iteration
        maxiter, epsl = 1000, 1e-3
        diff, i_count = 1.0, 0
        
        # Set Energies 
        oldE = np.zeros(NSP)
        newE = np.zeros(NSP)
        
        # Iterating alogrithm!
        while i_count < maxiter and diff > epsl:
            
            # Make Hartree-Fock Matrix
            MHF = np.zeros((NSP, NSP))
            
            sum_me2b = 0 
            
            processed_combinations = set()

            for i_alp in range(NSP):
                for i_bet in range(NSP):
                    if self.dict_SP[i_alp][1] != self.dict_SP[i_bet][1] : continue
                    sum_me2b = 0 
                    for i_gam in range(NSP):
                        for i_del in range(NSP):
                            # Check the conditions for matching SP values
                            if (self.dict_SP[i_alp][3]+self.dict_SP[i_gam][3] == self.dict_SP[i_bet][3]+self.dict_SP[i_del][3]) \
                               and (self.dict_SP[i_alp][4]+self.dict_SP[i_gam][4] == self.dict_SP[i_bet][4]+self.dict_SP[i_del][4]):
                                
                                # ignore np interaction
                                if self.dict_SP[i_alp][4] != -1*self.dict_SP[i_gam][4] : continue
                                if self.dict_SP[i_bet][4] != -1*self.dict_SP[i_del][4] : continue
                                    
                                # All possible permutations to check
                                permutations_to_check = [
                                    (i_alp, i_gam, i_bet, i_del),
                                    (i_alp, i_gam, i_del, i_bet),
                                    (i_gam, i_alp, i_del, i_bet),
                                    (i_gam, i_alp, i_bet, i_del)
                                ]

                                # Find the first valid permutation in dict_me2b
                                for perm in permutations_to_check:
                                    if perm in processed_combinations:
                                        continue

                                    # Check if this permutation exists in dict_me2b
                                    value = self.dict_me2b.get(perm, None)
                                    if value is not None:
                                        # Calculate the negative factor based on permutation number
                                        negative_factor = (-1) ** permutations_to_check.index(perm)

                                        # Add to sum with Rho and value
                                        sum_me2b += Rho[i_gam, i_del] * value #* negative_factor
                                        
                                        # Mark this combination as processed
                                        processed_combinations.add(perm)
                                        #print(sum_me2b)
                                        break
                    #print((SPH[i_alp] if i_alp == i_bet else 0))
                    MHF[i_alp,i_bet] = sum_me2b + (SPH[i_alp] if i_alp == i_bet else 0)
            
            #print(MHF)
            # Diagonalize and get the eigenstates of MHF
            Eeig, Coe = np.linalg.eigh(np.array(MHF))
            Rho = np.zeros((NSP, NSP))
            
            # print(Coe)
            # Update Rho
            for i_gam in range(NSP):
                for i_del in range(NSP):
                    Rho[i_gam, i_del] = sum(Coe[i_gam,i]*Coe[i_del,i] for i in list_A)
            
            # Get new Energies and calculate convergence
            newE     = np.array([e for e in Eeig])
            diff     = np.sum(np.abs(newE-oldE)/NSP)
            oldE     = newE
            i_count += 1
            
            # Print results
            
            #print("Single-particle energies, ordering may have changed")
            #for i in range(NSP):
            #    print('{0} {1:.4f}'.format(i,oldE[i]))
                
            #print('{0:.4f}'.format(sum(oldE[i] for i in range(self.A))))
            #print(str(i_count)+"th iteration")
        
        #print("iteration ended.")
        return sum(oldE[i] for i in list_A), oldE
        
    # Single Particle Hamiltonian (SPH)
    def onebody(self, SP_ind):
        e_gap = 41.5*np.power(self.A_c, -0.333)
        n, l  = self.dict_SP[SP_ind][0], self.dict_SP[SP_ind][1]
        return e_gap*(2*n + l + 3/2)
    

## Part 3: Set the initial value of me2b for code

In this block, it will generate initial ME2B as a list of (a,b,c,d,v), where using the method $\langle j_1 j_2 |V| j_1 j_2\rangle = BE(CS\pm 2 , j_1 j_2, J^\pi = 0^+) - BE(CS) -  \epsilon^{HF}_{j_1} - \epsilon^{HF}_{j_2}$, as given as <Shell Model from a Practitioner’s Point of View> by Hubert Grawe. 

This section needs to be refined again.

In [3]:
def generate_me2b(dict_SP, list_TB):
    BE_ds = pd.read_csv('atomicMassData.csv')
    e_gap = 41.5 * np.power(12, -1/3)
    A_core = 12
    Z_core = 6
    
    A_gap = {
        1.5: -2,
        0.5: 2
    }
    
    Z_gap = {
        1: 2,
        0: 1,
        -1: 0
    }
        
    Matrx_me2b = np.zeros((16, 16, 16, 16))
    list_me2b  = []
    
    for i in list_TB:
        for j in list_TB:
            if dict_SP[i[0]][3] + dict_SP[i[1]][3] != 0 or dict_SP[j[0]][3] + dict_SP[j[1]][3] != 0:
                continue
            # tz conservation
            if dict_SP[i[0]][4] + dict_SP[i[1]][4] != dict_SP[j[0]][4] + dict_SP[j[1]][4]:
                continue
            
            # Check if states are valid
            if dict_SP[i[0]][2] == 0:
                continue
            
            # Compute TBME with specific logic
            ji, jf, tzi = dict_SP[i[0]][2], dict_SP[j[0]][2], dict_SP[i[0]][4] + dict_SP[i[1]][4]
            
            if i == j:
                # Special calculation when i is the same as j
                try:
                    CSpm2 = BE_ds[(BE_ds['Z'] == Z_core + Z_gap.get(tzi)*A_gap.get(ji)/2) & 
                                  (BE_ds['A'] == A_core + A_gap.get(ji))]['BE'].tolist()[0]
                    CS = BE_ds[(BE_ds['Z'] == 6) & (BE_ds['A'] == 12)]['BE'].tolist()[0]
                    
                    if tzi == 0: e_gap = abs((A_core+A_gap.get(ji)/2)*(BE_ds[(BE_ds['Z'] == Z_core) & 
                                  (BE_ds['A'] == A_core + A_gap.get(ji))/2]['BE'].tolist()[0]+BE_ds[(BE_ds['Z'] == Z_core + Z_gap.get(tzi)*A_gap.get(ji)/2) & 
                                  (BE_ds['A'] == A_core + A_gap.get(ji))/2]['BE'].tolist()[0])-2*A_core*CS)
                    else: e_gap = abs(2*(A_core+A_gap.get(ji)/2)*BE_ds[(BE_ds['Z'] == Z_core + Z_gap.get(tzi)*A_gap.get(ji)/4) & 
                                  (BE_ds['A'] == A_core + A_gap.get(ji)/2)]['BE'].tolist()[0]-2*A_core*CS)
                    
                    
                    # Compute TBME
                    me2b = abs((A_core + A_gap.get(ji)) * CSpm2 - A_core * CS) - e_gap
                    me2b_value = me2b / 1000.
                    #print("tzi: "+str(tzi)+" / and tbme_value: "+str(me2b_value))
                
                except (IndexError, TypeError):
                    # Fallback if data lookup fails
                    me2b_value = 0
            else:
                # Random value centered at zero when i is not the same as j
                if dict_SP[i[0]][0] == dict_SP[j[0]][0]:
                    if ji == 1.5 and jf == 0.5: me2b_value = -0.8
                    elif ji == 0.5 and jf == 1.5: me2b_value = 0.46
                    elif ji == 1.5 and jf == 1.5: me2b_value = 0.34
                else: me2b_value = 0
                
            # Ensure symmetry for (a,b,c,d) and (c,d,a,b)
            Matrx_me2b[i[0]][i[1]][j[0]][j[1]] = me2b_value
            Matrx_me2b[j[0]][j[1]][i[0]][i[1]] = me2b_value
            
                
    
    # Collect TBME values
    for i in list_TB:
        for j in list_TB:
            list_me2b.append((i[0], i[1], j[0], j[1], Matrx_me2b[i[0]][i[1]][j[0]][j[1]]))
            #print(list_tbme[-1])
    
    return list_me2b

## Part 4: Set a function for optimization

In [4]:
def minimize_this(x):
    exp_energies = {
        '4He':-28.30,
        '12C':-92.16,
        '16O':-127.62
    }
        
    HF = HF_Machina()
    
    list_tbme = generate_me2b(HF.dict_SP, HF.list_TB)
    
    list_input_tbme = []
    
    for i, (a,b,c,d,_) in enumerate(list_tbme):
        list_input_tbme.append((a,b,c,d,x[i]))
    
    e_4He, _ = HF.Solve_HF(4,2,list_input_tbme)
    e_12C, _ = HF.Solve_HF(12,6,list_input_tbme)
    e_16O, _ = HF.Solve_HF(16,8,list_input_tbme)
    
    error = (e_4He - exp_energies['4He'])**2 + \
            (e_12C - exp_energies['12C'])**2 + \
            (e_16O - exp_energies['16O'])**2
    
    error = float(error)

    
    clear_output(wait=True)
    print("\n 4He energy : {0:.3f}".format(e_4He)+
          "\n 12C energy : {0:.3f}".format(e_12C)+
          "\n 16O energy : {0:.3f}".format(e_16O)+
          "\n error      : {0:.3f}".format(error)+
          "\n -------------------------", end='')
    
    
    return error


## Main

In [5]:
if __name__=='__main__':
    HF = HF_Machina()
    
    tbme0 = generate_me2b(HF.dict_SP, HF.list_TB)
    
    x0 = [t[4] for t in tbme0]
    
    print(HF.list_TB)

    for i_TB in HF.list_TB:
        print("state 1 :"+str(HF.dict_SP[i_TB[0]])+"   state 2 : "+str(HF.dict_SP[i_TB[1]]))
    
    #print(tbme0)
    e_12O, e_list_12O = HF.Solve_HF(16,8,tbme0)
    
    #callback.iteration = 0
    #result0 = minimize(minimize_this, x0, method='L-BFGS-B', tol=1e-6)
    #result1 = minimize(minimize_this, x0, method='CG', tol=1e-6)    
    result1 = minimize(minimize_this, x0, method='SLSQP', tol=1e-6)


    #result = optimize.fmin_cg(minimize_this, x0, disp=True)
    #result = sgd_optimization(tbme0)
    
    print(e_list_12O)
    print(e_12O)
    
    
    #tbme0


 4He energy : -28.300
 12C energy : -92.160
 16O energy : -127.620
 error      : 0.000
 -------------------------[26.76488899 26.76488899 27.21272299 27.21272299 44.90670432 44.90670432
 45.35453832 45.35453832 45.35453832 45.35453832 45.35453832 45.35453832
 48.72474332 48.72474332 48.72474332 48.72474332]
664.7948357876675


In [6]:
#resulta=[]
#for i, (a,b,c,d,_) in enumerate(tbme0):
#        resulta.append((a,b,c,d,result0.x[i]))
       
resultb=[]
for i, (a,b,c,d,_) in enumerate(tbme0):
        resultb.append((a,b,c,d,result1.x[i]))
"""        
resultc=[]
for i, (a,b,c,d,_) in enumerate(tbme0):
        resultc.append((a,b,c,d,result1.x[i]-result0.x[i]))
"""

'        \nresultc=[]\nfor i, (a,b,c,d,_) in enumerate(tbme0):\n        resultc.append((a,b,c,d,result1.x[i]-result0.x[i]))\n'

In [7]:
resultb

[(2, 3, 2, 3, -10.236273999999996),
 (2, 3, 11, 12, -0.799999999999987),
 (2, 3, 1, 2, 8.495157864754263e-16),
 (2, 3, 0, 3, 2.811557528216948e-15),
 (2, 3, 4, 13, -3.854668419609562e-15),
 (2, 3, 0, 1, -5.037005140072436e-15),
 (2, 3, 5, 12, -2.9150817177950396e-15),
 (2, 3, 7, 10, -1.5289606265101927e-15),
 (2, 3, 8, 9, 0.0),
 (2, 3, 10, 13, -0.8),
 (2, 3, 5, 6, 0.0),
 (2, 3, 9, 14, 0.0),
 (2, 3, 14, 15, 0.46),
 (2, 3, 8, 15, 0.0),
 (2, 3, 6, 11, 0.0),
 (2, 3, 4, 7, 0.0),
 (11, 12, 2, 3, -0.8),
 (11, 12, 11, 12, 6.756037999999986),
 (11, 12, 1, 2, 0.0),
 (11, 12, 0, 3, 0.0),
 (11, 12, 4, 13, 0.0),
 (11, 12, 0, 1, 0.0),
 (11, 12, 5, 12, 0.0),
 (11, 12, 7, 10, 0.0),
 (11, 12, 8, 9, 0.0),
 (11, 12, 10, 13, 0.34),
 (11, 12, 5, 6, 0.0),
 (11, 12, 9, 14, 0.0),
 (11, 12, 14, 15, 0.46),
 (11, 12, 8, 15, 0.0),
 (11, 12, 6, 11, 0.0),
 (11, 12, 4, 7, 0.0),
 (1, 2, 2, 3, 0.0),
 (1, 2, 11, 12, 0.0),
 (1, 2, 1, 2, -68.57544563079901),
 (1, 2, 0, 3, 0.46),
 (1, 2, 4, 13, -0.8),
 (1, 2, 0, 1, 0.0),


In [25]:
e_15O, oldE = HF.Solve_HF(12,6,resultb)

In [26]:
e_15O

-92.15998059825665

In [27]:
oldE

array([-61.31966576, -61.3194896 , -61.3194896 , -61.3194896 ,
       -41.36272264, -41.36272264,  27.21272299,  27.21272299,
        45.35453832,  45.35453832,  45.35453832,  45.35453832,
        45.35453832,  45.35453832,  45.35453832,  45.35453832])

In [11]:
e_12O, e_list_12O = HF.Solve_HF(12,6,tbme0)
e_list_12O = e_list_12O - 35.92
print(e_list_12O)
print(e_12O)

[-9.15511101 -9.15511101 -8.70727701 -8.70727701  8.98670432  8.98670432
  9.43453832  9.43453832  9.43453832  9.43453832  9.43453832  9.43453832
  9.43453832  9.43453832 12.80474332 12.80474332]
469.8958625133153


In [12]:
print(HF.dict_SP)

{0: [0, 0, 0.5, -0.5, -0.5], 1: [0, 0, 0.5, 0.5, -0.5], 2: [0, 0, 0.5, -0.5, 0.5], 3: [0, 0, 0.5, 0.5, 0.5], 4: [0, 1, 1.5, -1.5, -0.5], 5: [0, 1, 1.5, -0.5, -0.5], 6: [0, 1, 1.5, 0.5, -0.5], 7: [0, 1, 1.5, 1.5, -0.5], 8: [0, 1, 0.5, -0.5, -0.5], 9: [0, 1, 0.5, 0.5, -0.5], 10: [0, 1, 1.5, -1.5, 0.5], 11: [0, 1, 1.5, -0.5, 0.5], 12: [0, 1, 1.5, 0.5, 0.5], 13: [0, 1, 1.5, 1.5, 0.5], 14: [0, 1, 0.5, -0.5, 0.5], 15: [0, 1, 0.5, 0.5, 0.5]}
